In [ ]:
from datetime import datetime
import logging
import random
import time

from bs4 import BeautifulSoup
import psycopg2
from selenium import webdriver

def log():
    formatter = logging.Formatter('%(asctime)s - [%(levelname)s] - %(message)s')
    hdlr = logging.StreamHandler()
    hdlr.setFormatter(formatter)
    logger = logging.getLogger() 
    logger.setLevel(logging.INFO)
    logger.addHandler(hdlr)
    return logger

class Initial:
    def __init__(self):
        self.url_pool = {'info':'https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID='
                         ,'YoY':'https://goodinfo.tw/StockInfo/StockDividendPolicy.asp?STOCK_ID='
                         ,'price':'https://www.twse.com.tw/exchangeReport/STOCK_DAY_AVG?response=html&date=&stockNo='
                        }
        self.driver = webdriver.Chrome('G:\Anaconda\Tools\chromedriver.exe')
        
class Flag(Initial):
    
    def __init__(self):
        self.id = 0
        super(Flag, self).__init__()
        
    def search(self):
        logger.info('Start to search stock id.')
        for key, value in self.url_pool.items():
            if key == 'info':
                list_sql = "select stock_id from stock_info where flag in ('1','2');"
                __db = Database()
                exist = __db.get(list_sql)
                while self.id < 10000:
                    self.id += 1
                    s_id = str(self.id).zfill(4)
                    if s_id not in exist[0]:
                        url = value + s_id
                        logger.info('parse {}'.format(url))
                        self.driver.get(url)
                        soup = BeautifulSoup(self.driver.page_source,'lxml')
                        hk_sql = "delete from stock_info where stock_id = '{}';".format(s_id)
                        try:
                            name = soup.select('.std_tbl tr > td')[1].text.split('\xa0')[1]
                            dml_sql = "insert into stock_info (stock_id, company, flag) values('{}','{}',3);".format(s_id,name)
                        except:
                            dml_sql = "insert into stock_info (stock_id, company, flag) values('{}',null,0);".format(s_id)
                
                        try:
                            __db.run(hk_sql)
                            __db.run(dml_sql)
                            __db.logout()
                        except:
                            logger.error('Database error')
                        
                        if '瀏覽量異常' in str(soup):
                            f.note(self.id)
                            break
                        
                        t = random.randint(30,60)
                        time.sleep(t+120)
            
        self.driver.quit()
    
    @staticmethod
    def note(count):
        try:
            __n = Database()
            if count == 0:
                __n.run("delete from check_blocking;")
                note_sql = "insert into check_blocking select 'start', current_timestamp;"
                __n.run(note_sql)
            else:
                note_sql = "insert into check_blocking select '{}', current_timestamp;".format(count)
                __n.run(note_sql)

            __n.logout()
        except:
            logger.error('Database error')
            
class Crawler(Initial):
    
    def __init__(self):
        super(Crawler, self).__init__()
        self.db = Database()
        self.id_pool = self.db.get("select stock_id, company from stock_info where flag='1' order by stock_id asc;")
        
    def yoy(self):
        for key, value in self.url_pool.items():
            if key == 'YoY':
                logger.info('Start to parse YoY information of stocks.')
                for number in self.id_pool:
                    stock_id = number[0]
                    url = value + stock_id
                    logger.info('parse {}'.format(url))
                    __e = Extract(url)
                    
                    per = __e.data('.std_tbl > tbody > tr:nth-of-type(5) > td:nth-of-type(6)', 'float','a')
                    pbr = __e.data('.std_tbl > tbody > tr:nth-of-type(5) > td:nth-of-type(7)', 'float','a')
                    self.db.run("delete from stock_per_pbr where stock_id = '{}';".format(stock_id))
                    self.db.run("insert into stock_per_pbr values('{}', {}, {});".format(stock_id, per, pbr))
                    
                    for count in range(1,5):
                        cash_dividend = __e.data('.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                         > td:nth-of-type(4)'.format(count), 'float','a')
                        stock_dividend = __e.data('.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                         > td:nth-of-type(7)'.format(count), 'float','a')
                        eps = __e.data('.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                               > td:nth-of-type(23)'.format(count), 'float','a')
                        year = __e.data('.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                > td:nth-of-type(22)'.format(count))
                        spls_aption = __e.data('.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                       > td:nth-of-type(26)'.format(count), 'float','a')
                        self.db.run("delete from stock_yoy_revenue where stock_id = '{}' and year = '{}';".format(stock_id, year))
                        self.db.run("insert into stock_yoy_revenue values('{}','{}',\
                              {},{},{},{})".format(stock_id, year, eps, stock_dividend, cash_dividend, spls_aption))
                                    
                    t = random.randint(30,60)
                    time.sleep(t+180)
                
                self.db.logout()
                logger.info('The parsing of stocks\' YoY information finished.')

        self.driver.quit()
    
    def price(self):
        
        for pri_key, pri_value in self.url_pool.items():
            if pri_key == 'price':
                logger.info('Start to parse price of stocks.')
                for pri_number in self.id_pool:
                    stock_no = pri_number[0]
                    pri_date = 'date=' + datetime.strftime(datetime.today(),'%Y%m%d')
                    pri_url = pri_value.replace('date=',pri_date) + stock_no
                    logger.info('parse {}'.format(pri_url))
                    __e = Extract(pri_url)
                    trading_day = __e.data('table > tbody > tr > td:nth-of-type(1)')
                    
                    for c in range(0, len(trading_day)-1):
                        ad = __e.data('table > tbody > tr > td:nth-of-type(1)', 'convert_to_AD', 'b', c)
                        p = __e.data('table > tbody > tr > td:nth-of-type(2)', 'float', 'b', c)
                        self.db.run("delete from stock_price where stock_id='{}' and trading_day='{}';".format(stock_no, ad))
                        self.db.run("insert into stock_price values('{}','{}',{});".format(stock_no, ad, p))
                    
                    t = random.randint(30,60)
                    time.sleep(t+150)
                
                self.db.logout()
                logger.info('The parsing of stocks\' price finished.')
        
        self.driver.quit()

class Extract(Initial):
    
    def __init__(self, url):
        super(Extract, self).__init__()
        self.driver.get(url)
        self.soup = BeautifulSoup(self.driver.page_source,'lxml')
    
    def data(self, selector, transfer=None, type=None, count=None):
        if transfer == 'float' and type == 'a':
            try:
                return float(self.soup.select(selector)[0].text)
            except:
                return 'null'
        elif transfer == 'float' and type == 'b':
            try:
                return float(self.soup.select(selector)[count].text)
            except:
                return -1
        elif transfer == 'convert_to_AD':
            try:
                ad = self.soup.select(selector)[count].text
                pat = ad.split('/')[0]
                new_pat = str(int(pat) + 1911)
                return ad.replace(pat, new_pat)
            except:
                return '9999'
        else:
            return self.soup.select(selector)        
                
class Database:
    
    def __init__(self):
        self.con = psycopg2.connect(database='stock', user='postgres', password='postgres', host='localhost', port='5432')
        self.cur = self.con.cursor()
    
    def run(self, sql):
        self.cur.execute(sql)
        self.con.commit()
        
    def get(self, sql):
        self.cur.execute(sql)
        data = self.cur.fetchall()
        self.con.commit()
        return data   # return tuple
    
    def logout(self):
        try:
            self.con.close()
        except:
            logger.info('already logged out!')

if __name__ == '__main__':
    logger = log()
    c = Crawler()
    c.yoy()
    